# 3d mitochondria visualizer using vtk
This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409611,2864896,3878784,93134,59860,836,93000,59406,798,93294,60294,867,648518346349536888,333419.72,214298.80,33440
2409612,2161774,4075748,80312,56522,1814,79536,55348,1714,80940,57054,1889,648518346349524070,287516.96,202348.76,72560
2409613,2753701,4576444,90324,60436,2037,89490,60120,1851,90750,61138,2142,648518346341354380,323359.92,216360.88,81480
2409614,1963708,5805612,75674,72546,744,75266,71628,670,76032,73302,795,648518346343047176,270912.92,259714.68,29760


# Enter a cellid of interest

In [6]:
# pull all mitos from a cellid of interest
cellid = 648518346349537555

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf

# interesting cellids
# 648518346349530724 microglia with 149 mitos
# 648518346349527319 astrocyte containing the largest mito in the volume and several additional large mitos
# 648518346349537555 apical dendrite likely from a L5 or L6 pyr neuron
# 648518346349537741 pyr neuron with largest mito in the volume for a neuron
# 648518346349508279 oligodendrocyte

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
10849,2703814,13149480,93034,50892,922,90438,37016,21,96796,66074,1410,648518346349537555,333061.72,182193.36,36880
37770,3230066,115228,98264,75898,1683,98118,75608,1674,98408,76120,1698,648518346349537555,351785.12,271714.84,67320
40863,3149741,1109944,99442,41520,1094,98996,40024,1075,99842,43142,1131,648518346349537555,356002.36,148641.60,43760
66804,2838061,60384,92840,49246,1043,92790,48964,1041,92914,49476,1048,648518346349537555,332367.20,176300.68,41720
89547,3102126,1352,96986,66392,1393,96976,66380,1391,96998,66410,1396,648518346349537555,347209.88,237683.36,55720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330161,2832798,204600,92022,46360,1027,91882,46164,1021,92246,46624,1035,648518346349537555,329438.76,165968.80,41080
2345385,3101683,911076,97016,68530,1463,96518,66132,1378,97396,70632,1552,648518346349537555,347317.28,245337.40,58520
2348717,3087433,235844,96070,60064,1230,95952,59800,1223,96148,60316,1236,648518346349537555,343930.60,215029.12,49200
2357417,3225167,611048,97960,73854,1619,97722,72438,1575,98196,75128,1663,648518346349537555,350696.80,264397.32,64760


In [7]:
mito_query_mitolist = mito_querydf.mito_id.to_list()
print(f"length: "+str(len(mito_query_mitolist)))
print(f"type: "+str(type(mito_query_mitolist)))
print('')
# print(mito_query_mitolist) # uncomment to print the mito id list

length: 87
type: <class 'list'>



# vtk 3d viewer

In [8]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://seungdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [9]:
cell_id = cellid

In [10]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [11]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [12]:
len(mito_query_mitolist)

87

In [13]:
mito_query_mitolist[0]

2703814

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [14]:
# download the mito meshes for this cell

for i in range(len(mito_query_mitolist)):
    mito_id = mito_query_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

In [15]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator = {}
for i in range(len(mito_query_mitolist)):
    mito_seg_id = mito_query_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator)

In [16]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.075
cell_color = (0.2, 0.4, 0.7)
mito_opac = 0.35

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator = {}
for i in range(len(mito_query_mitolist)):
    random_color = list(np.random.random(size=3))
    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
locals().update(actor_iterator)

# update dictionary for the render actors code below
actor_iterator['mesh_actor'] = mesh_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=150)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x00000207EA9762A0) at 0x000002078317F468>